In [2]:
import numpy as np
import cv2
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import itertools

In [3]:
def channel_norm(img):
    mean = np.mean(img)
    std = np.std(img)
    pixels = (img-mean)/(std)
    return pixels

In [45]:
def parse_line(line):
    line_parts = line.tolist()
    img_name = line_parts[0]
    #rect = list(map(int, list(map(float, line_parts[1:5]))))
    rect = list(map(int, line_parts[1:5]))
    landmarks = list(map(float, line_parts[5: len(line_parts)]))
    return img_name, rect, landmarks

In [6]:
class Normalize(object):
    """
        Resieze to train_boarder x train_boarder. Here we use 112 x 112
        Then do channel normalization: (image - mean) / std_variation
    """
    def __call__(self,sample):
        img, landmask = sample['image'], sample['landmask']
        img_resize = np.asarray(img.resize((train_boarder, train_boarder),Image.BILINEAR))
        img_resize = channel_norm(img_resize)
        return {'image':img_resize,'landmask':landmask}

In [10]:
class ToTensor(object):
    """
        Convert ndarrays in sample to Tensors.
        Tensors channel sequence: N x C x H x W
    """
    def __call__(self,sample):
        '''
        numpy img: H*W*C
        torch.tensorimg: N*C*H*W
        '''
        img, landmask = sample['image'], sample['landmask']
        img = img.transpose((2, 0, 1))
        img = np.expand_dims(img, axis=0)
        return {'image':torch.from_numpy(img), 'landmask':torch.from_numpy(landmask)}

In [ ]:
class FaceLandmarksDataset():
    def __init__(self, data, transforms= None)：
        '''
        :param lines: src_line
        :param transform: data transform
        '''
        self.data = data
        self.transforms = transforms
    
    def __len__(self):
        return len(data)
    
    def __getitem__(self, idx):
        img, rect, landmasks = parse_line(self.data.values[idx])
        #转为灰度图
        img = Image.open(img).convert('L')
        img = img.crop(tuple(rect))
        landmasks = np.array(landmasks).astype('float32')
        '''
        undone
        '''
        sample = {'image':img, 'landmasks':landmasks}
        sample = self.transform(sample)
        return sample

In [8]:
df = pd.read_csv('F:/data/cv_learn/项目/项目二/train/train_annotation.csv')

In [1]:
df

NameError: name 'df' is not defined